In [1]:
from TSPGraph import Graph
import numpy as np
from functools import lru_cache
import copy
from tqdm import tqdm
from multiprocessing import Pool

In [2]:
graph = Graph('A')

In [3]:
def random_solution():
    sol = np.random.choice(np.arange(200),size=100,replace=False)
    return np.append(sol,sol[0])

In [4]:
combination_pairs = [(x,y) for x in range(1,100) for y in range(x+2, 100)]
combination_pairs_edges = [(x,y) for x in range(1,100) for y in range(x+2, 100)]

In [5]:
def ExchangeVertices(solution, i, j):
    result = copy.deepcopy(solution)
    result[[i,j]] = result[[j,i]]
    return result

def ExchangeVerticesDelta(graph, solution, i, j):
    res = (graph.dist_matrix[solution[i-1], solution[j]] + graph.dist_matrix[solution[j], solution[i+1]] + graph.dist_matrix[solution[j-1], solution[i]] + graph.dist_matrix[solution[i], solution[j+1]]) - (graph.dist_matrix[solution[i-1], solution[i]] + graph.dist_matrix[solution[i], solution[i+1]] + graph.dist_matrix[solution[j-1], solution[j]] + graph.dist_matrix[solution[j], solution[j+1]])
    # check = graph.cycle_cost(ExchangeVertices(solution,i,j)) - graph.cycle_cost(solution)
    # print(res)
    # print(check)
    # print(solution)
    # print(ExchangeVertices(solution,i,j))
    # assert check == res
    return res

def MinInAllExchangedVertices(graph, solution):
    minDelta = 0
    resultExchange = tuple()
    for x in combination_pairs:
        delta = ExchangeVerticesDelta(graph, solution, x[0], x[1])
        if delta < minDelta:
            minDelta = delta
            resultExchange = (x[0], x[1])
    return (minDelta, resultExchange)

def GreedyExchangeVertex(graph, solution):
    for x in combination_pairs:
        delta = ExchangeVerticesDelta(graph, solution, x[0], x[1])
        if delta < 0:
            return (x[0], x[1])
    return None


MinInAllExchangedVertices(graph, random_solution())
# ExchangeVerticesDelta(graph, random_solution(), 1, 5)

(-9073, (30, 43))

In [6]:
def ExchangeEdges(solution, i, j):
    assert i<j
    result = copy.deepcopy(solution)
    # print(i,j)
    # print(result[:i])
    # print(result[j:i-1:-1])
    # print(result[j+1:])
    result = np.concatenate((result[:i], result[j:i-1:-1], result[j+1:]))
    return result

def ExchangeEdgesDelta(graph, solution, i, j):
    assert i<j
    res = (graph.dist_matrix[solution[i-1], solution[j]] + graph.dist_matrix[solution[i], solution[j+1]]) - (graph.dist_matrix[solution[i-1], solution[i]] + graph.dist_matrix[solution[j], solution[j+1]])
    # check = graph.cycle_cost(ExchangeEdges(solution,i,j)) - graph.cycle_cost(solution)
    # print(res)
    # print(check)
    # print(solution)
    # print(ExchangeEdges(solution,i,j))
    # assert res == check
    return res

def MinInAllExchangedEdges(graph, solution):
    minDelta = 0
    resultExchange = tuple()
    for x in combination_pairs_edges:
        delta = ExchangeEdgesDelta(graph, solution, x[0], x[1])
        if delta < minDelta:
            minDelta = delta
            resultExchange = (x[0], x[1])
    return (minDelta, resultExchange)

def GreedyExchangeEdges(graph, solution):
    for x in combination_pairs_edges:
        delta = ExchangeEdgesDelta(graph, solution, x[0], x[1])
        if delta < 0:
            return (x[0], x[1])
    return None

MinInAllExchangedEdges(graph, random_solution())

(-5377, (34, 78))

In [7]:

def SwapVertexWithNewOne(solution, i, a):
    result = copy.deepcopy(solution)
    result[i] = a
    return result

def SwapVertexWithNewOneDelta(graph, solution, i, a):
    return (graph.objective[solution[i-1], a] + graph.objective[a, solution[i+1]]) - (graph.objective[solution[i-1], solution[i]] + graph.objective[solution[i], solution[i+1]])

def MinInAllSwaps(solution, graph):
    domain = np.array(range(200))
    not_in_sol = domain[np.isin(domain, solution, invert=True)]
    #pairs of index in current cycle - node 
    pairs = np.transpose([np.tile(domain[:100], 100), np.repeat(not_in_sol, 100)])
    minDelta = 0
    resultSwap = tuple()
    for x in pairs:
        delta = SwapVertexWithNewOneDelta(graph, solution, x[0], x[1])
        if delta < minDelta:
            minDelta = delta
            resultSwap = (x[0], x[1])
    return (minDelta, resultSwap)

def GreedySwap(solution, graph):
    domain = np.array(range(200))
    not_in_sol = domain[np.isin(domain, solution, invert=True)]
    #pairs of index in current cycle - node 
    pairs = np.transpose([np.tile(domain[:100], 100), np.repeat(not_in_sol, 100)])
    for x in pairs:
        delta = SwapVertexWithNewOneDelta(graph, solution, x[0], x[1])
        if delta < 0:
            return (x[0], x[1])
    return None
print(MinInAllSwaps(random_solution(), graph))

(-7446, (34, 185))


In [8]:
def SteepestLocalSearch(graph, initial_solution, type):
    current_solution = initial_solution
    current_cost = graph.cycle_cost(current_solution)
    max_iter = 1_000_000
    while(max_iter > 0):
        # print(current_cost)
        best_swap_val, best_swap = MinInAllSwaps(current_solution, graph)
        if type == 'edge':
            best_exchange_val, best_exchange = MinInAllExchangedEdges(graph, current_solution)
        else:
            best_exchange_val, best_exchange = MinInAllExchangedVertices(graph, current_solution)
        
        if best_swap_val >= 0 and best_exchange_val >= 0:
            break

        if best_swap_val < best_exchange_val:
            current_solution = SwapVertexWithNewOne(current_solution, best_swap[0], best_swap[1])
            # print('swap' + str(best_swap_val))
        elif type == 'edge':
            current_solution = ExchangeEdges(current_solution, best_exchange[0], best_exchange[1])
            # print('exchange ' + str(best_exchange_val) + str(best_exchange))
        else:
            current_solution = ExchangeVertices(current_solution, best_exchange[0], best_exchange[1])
            # print('exchange ' + str(best_exchange_val) + str(best_exchange))

        max_iter -= 1
        # current_cost = graph.cycle_cost(current_solution)
    return graph.cycle_cost(current_solution)
        
SteepestLocalSearch(graph, random_solution(), 'edge')

77654

In [9]:
def GreedyLocalSearch(graph, initial_solution, type):
    current_solution = initial_solution
    current_cost = graph.cycle_cost(current_solution)
    max_iter = 1_000_000
    while(max_iter > 0):
        # print(current_cost)
        max_iter -= 1

        swap = GreedySwap(current_solution, graph)
        if swap is not None:
            current_solution = SwapVertexWithNewOne(current_solution, swap[0], swap[1])
            continue
        if type == 'edge':
            exchange = GreedyExchangeEdges(graph, current_solution)
            if exchange is not None:
                current_solution = ExchangeEdges(current_solution, exchange[0], exchange[1])
                continue
        else:
            exchange = GreedyExchangeVertex(graph, current_solution)
            if exchange is not None:
                current_solution = ExchangeVertices(current_solution, exchange[0], exchange[1])
                continue
        break

        
        # current_cost = graph.cycle_cost(current_solution)
    return graph.cycle_cost(current_solution)
        
GreedyLocalSearch(graph, random_solution(), 'edge')

c:\Users\Kuba\AppData\Local\Programs\Python\Python39\lib\site-packages\numpy\core\_asarray.py:83: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray
  return array(a, dtype, copy=False, order=order)


126865

In [15]:
random_solutions = [random_solution() for i in range(200)]
previous_solutions_D = np.load('Data/WeightedRegretResults/d_weighted_regret_all.npy')
previous_solutions_C = np.load('Data/WeightedRegretResults/c_weighted_regret_all.npy')
previous_solutions_B = np.load('Data/WeightedRegretResults/b_weighted_regret_all.npy')
previous_solutions_A = np.load('Data/WeightedRegretResults/a_weighted_regret_all.npy')

In [11]:
def SteepestExperiment(graph, init_solutions, type):
    steep_random_edge = []
    for i in tqdm(range(200)):
        steep_random_edge.append(SteepestLocalSearch(graph, init_solutions[i], type))
    return np.array(steep_random_edge)

In [12]:
def GreedyExperiment(graph, init_solutions, type):
    steep_random_vertex = []
    for i in tqdm(range(200)):
        steep_random_vertex.append(GreedyLocalSearch(graph, init_solutions[i], type))
    return np.array(steep_random_vertex)

In [13]:
def printResults(x):
    print("min: {} max: {} avg: {}".format(np.min(x), np.max(x), np.mean(x)))

In [14]:
res = GreedyExperiment(Graph('D'), random_solutions, 'edge')
print('greedy random edge')
printResults(res)
res = GreedyExperiment(Graph('D'), random_solutions, 'vert')
print('greedy random vertex')
printResults(res)
res = GreedyExperiment(Graph('D'), previous_solutions_D, 'edge')
print('greedy previous edge')
printResults(res)
res = GreedyExperiment(Graph('D'), previous_solutions_D, 'vert')
print('greedy previous vertex')
printResults(res)

res = SteepestExperiment(Graph('D'), random_solutions, 'edge')
print('steepest random edge')
printResults(res)
res = SteepestExperiment(Graph('D'), random_solutions, 'vert')
print('steepest random vertex')
printResults(res)
res = SteepestExperiment(Graph('D'), previous_solutions_D, 'edge')
print('steepest previous edge')
printResults(res)
res = SteepestExperiment(Graph('D'), previous_solutions_D, 'vert')
print('steepest previous vertex')
printResults(res)

  0%|          | 0/200 [00:00<?, ?it/s]

100%|██████████| 200/200 [09:57<00:00,  2.99s/it]


greedy random edge
min: 45869 max: 77418 avg: 66306.375


100%|██████████| 200/200 [08:44<00:00,  2.62s/it]


greedy random vertex
min: 54568 max: 100416 avg: 81553.35


...

steepest random vertex
min: 59343 max: 83020 avg: 67301.335


100%|██████████| 200/200 [01:09<00:00,  2.88it/s]


steepest previous edge
min: 46670 max: 54971 avg: 51683.78


100%|██████████| 200/200 [01:00<00:00,  3.29it/s]

steepest previous vertex
min: 46670 max: 55136 avg: 51874.05


In [16]:
res = GreedyExperiment(Graph('C'), random_solutions, 'edge')
print('greedy random edge')
printResults(res)
res = GreedyExperiment(Graph('C'), random_solutions, 'vert')
print('greedy random vertex')
printResults(res)
res = GreedyExperiment(Graph('C'), previous_solutions_C, 'edge')
print('greedy previous edge')
printResults(res)
res = GreedyExperiment(Graph('C'), previous_solutions_C, 'vert')
print('greedy previous vertex')
printResults(res)

res = SteepestExperiment(Graph('C'), random_solutions, 'edge')
print('steepest random edge')
printResults(res)
res = SteepestExperiment(Graph('C'), random_solutions, 'vert')
print('steepest random vertex')
printResults(res)
res = SteepestExperiment(Graph('C'), previous_solutions_C, 'edge')
print('steepest previous edge')
printResults(res)
res = SteepestExperiment(Graph('C'), previous_solutions_C, 'vert')
print('steepest previous vertex')
printResults(res)

  0%|          | 0/200 [00:00<?, ?it/s]c:\Users\Kuba\AppData\Local\Programs\Python\Python39\lib\site-packages\numpy\core\_asarray.py:83: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray
  return array(a, dtype, copy=False, order=order)
100%|██████████| 200/200 [06:41<00:00,  2.01s/it]


greedy random edge
min: 49509 max: 79017 avg: 68856.285


100%|██████████| 200/200 [06:55<00:00,  2.08s/it]


greedy random vertex
min: 57023 max: 103052 avg: 84826.455


100%|██████████| 200/200 [00:47<00:00,  4.17it/s]


...

steepest random vertex
min: 59496 max: 79880 avg: 68450.065


100%|██████████| 200/200 [01:17<00:00,  2.57it/s]


steepest previous edge
min: 51931 max: 58196 avg: 54326.935


100%|██████████| 200/200 [01:14<00:00,  2.68it/s]

steepest previous vertex
min: 52070 max: 58196 avg: 54594.045


In [17]:
res = GreedyExperiment(Graph('B'), random_solutions, 'edge')
print('greedy random edge')
printResults(res)
res = GreedyExperiment(Graph('B'), random_solutions, 'vert')
print('greedy random vertex')
printResults(res)
res = GreedyExperiment(Graph('B'), previous_solutions_B, 'edge')
print('greedy previous edge')
printResults(res)
res = GreedyExperiment(Graph('B'), previous_solutions_B, 'vert')
print('greedy previous vertex')
printResults(res)

res = SteepestExperiment(Graph('B'), random_solutions, 'edge')
print('steepest random edge')
printResults(res)
res = SteepestExperiment(Graph('B'), random_solutions, 'vert')
print('steepest random vertex')
printResults(res)
res = SteepestExperiment(Graph('B'), previous_solutions_B, 'edge')
print('steepest previous edge')
printResults(res)
res = SteepestExperiment(Graph('B'), previous_solutions_B, 'vert')
print('steepest previous vertex')
printResults(res)

100%|██████████| 200/200 [07:21<00:00,  2.21s/it]


greedy random edge
min: 68934 max: 128097 avg: 110734.53


100%|██████████| 200/200 [07:48<00:00,  2.34s/it]


greedy random vertex
min: 79165 max: 153401 avg: 127430.97


100%|██████████| 200/200 [01:18<00:00,  2.55it/s]


...

steepest random vertex
min: 77477 max: 123748 avg: 91954.505


100%|██████████| 200/200 [01:34<00:00,  2.12it/s]


steepest previous edge
min: 68111 max: 77153 avg: 71617.52


100%|██████████| 200/200 [01:42<00:00,  1.95it/s]

steepest previous vertex
min: 68826 max: 77546 avg: 72020.725


In [18]:
res = GreedyExperiment(Graph('A'), random_solutions, 'edge')
print('greedy random edge')
printResults(res)
res = GreedyExperiment(Graph('A'), random_solutions, 'vert')
print('greedy random vertex')
printResults(res)
res = GreedyExperiment(Graph('A'), previous_solutions_A, 'edge')
print('greedy previous edge')
printResults(res)
res = GreedyExperiment(Graph('A'), previous_solutions_A, 'vert')
print('greedy previous vertex')
printResults(res)

res = SteepestExperiment(Graph('A'), random_solutions, 'edge')
print('steepest random edge')
printResults(res)
res = SteepestExperiment(Graph('A'), random_solutions, 'vert')
print('steepest random vertex')
printResults(res)
res = SteepestExperiment(Graph('A'), previous_solutions_A, 'edge')
print('steepest previous edge')
printResults(res)
res = SteepestExperiment(Graph('A'), previous_solutions_A, 'vert')
print('steepest previous vertex')
printResults(res)

100%|██████████| 200/200 [06:13<00:00,  1.87s/it]


greedy random edge
min: 75868 max: 129900 avg: 114601.405


100%|██████████| 200/200 [06:50<00:00,  2.05s/it]


greedy random vertex
min: 85042 max: 153466 avg: 131795.885


100%|██████████| 200/200 [00:37<00:00,  5.38it/s]


...

steepest random vertex
min: 87107 max: 128900 avg: 96927.06


100%|██████████| 200/200 [00:55<00:00,  3.63it/s]


steepest previous edge
min: 74464 max: 79628 avg: 75752.57


100%|██████████| 200/200 [00:43<00:00,  4.61it/s]

steepest previous vertex
min: 74733 max: 79906 avg: 75961.205
